In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install python-telegram-bot --upgrade
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [10]:
# Импорты
import os
import shutil
import nest_asyncio
from pathlib import Path
from telegram import Update, KeyboardButton, ReplyKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters, CallbackContext
from ultralytics import YOLO

# Применяем патч для корректной работы asyncio в Colab
nest_asyncio.apply()

# ==== Настройки ====
MODEL_PATH = "/content/best.pt"  # Убедитесь, что модель здесь
BOT_TOKEN = "7571479193:AAGFcr_dXla0aRgnFpEmIhxKFuLnm1CQrkg"  # Замените на ваш токен

# Проверка наличия модели
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Модель не найдена по пути: {MODEL_PATH}")

# Загрузка модели
model = YOLO(MODEL_PATH)

# ==== Клавиатура ====
def get_main_keyboard():
    keyboard = [
        [KeyboardButton("📸 Отправить фото"), KeyboardButton("ℹ️ О боте")],
        [KeyboardButton("❓ Помощь"), KeyboardButton("⚙️ Настройки")]
    ]
    return ReplyKeyboardMarkup(keyboard, resize_keyboard=True)

# ==== Обработчики команд ====
async def about_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    about_text = "🔍 Этот бот использует нейросетевую модель YOLOv8 для обнаружения:\n" \
                 "- ⚔️ Четырех типов боевых ножей (армейских, охотничьих, сабли, складных)\n\n" \
                 "🛡️ Обучена на рентгеновских снимках с ИДК (инспекционно-досмотрового комплекса)\n" \
                 "🤖 Точность: 85% на тестовой выборке\n" \
                 "🎓 Часть дипломного проекта по компьютерному зрению"
    await update.message.reply_text(about_text)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    welcome_text = "🤖 Добро пожаловать в бота обнаружения боевых ножей!\n\n" \
                   "📷 Отправьте рентгеновский снимок с ИДК или воспользуйтесь кнопками меню.\n" \
                   "🔍 Я помогу найти опасные предметы на изображении."
    await update.message.reply_text(welcome_text, reply_markup=get_main_keyboard())

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = "📌 Инструкция:\n" \
                "1. Отправьте фото с ИДК (инспекционно-досмотрового комплекса)\n" \
                "2. Подождите несколько секунд\n" \
                "3. Получите результат с выделенными боевыми ножами\n\n" \
                "⚠️ Работает только с рентгеновскими снимками грузов\n" \
                "🛃 Используется в таможенном контроле для досмотра без вскрытия"
    await update.message.reply_text(help_text)

async def settings_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    settings_text = "⚙️ Настройки:\n" \
                    "▫️ Уровень уверенности: 85%\n" \
                    "▫️ Размер изображения: 640x640 px\n" \
                    "▫️ Тип изображения: Рентгеновский снимок с ИДК\n" \
                    "▫️ Поддержка форматов: JPG, PNG"
    await update.message.reply_text(settings_text)

async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        # Показываем статус "печатает"
        await context.bot.send_chat_action(chat_id=update.effective_message.chat_id, action="typing")

        # Создаем временную папку
        temp_dir = Path("/content/temp")
        temp_dir.mkdir(exist_ok=True)

        # Получаем фото
        photo = update.message.photo[-1]
        file = await photo.get_file()

        # Сохраняем изображение
        input_path = temp_dir / "input.jpg"
        await file.download_to_drive(custom_path=str(input_path))

        # Детекция
        results = model.predict(
            source=str(input_path),
            save=True,
            imgsz=640,
            conf=0.25
        )

        # Путь к результату
        output_dir = Path("/content/runs/detect/predict")
        output_files = list(output_dir.glob("*.jpg"))

        # Показываем статус "отправляет фото"
        await context.bot.send_chat_action(chat_id=update.effective_message.chat_id, action="upload_photo")

        if output_files:
            output_path = output_files[0]
            caption = "✅ Обнаружены опасные предметы!\n" \
                     "🔍 Результаты анализа:\n" \
                     "- Проверено изображение\n" \
                     "- Найдены потенциально опасные объекты\n" \
                     "- Применены методы компьютерного зрения"
            await update.message.reply_photo(
                photo=open(output_path, 'rb'),
                caption=caption
            )
        else:
            await update.message.reply_text("❌ На изображении ничего не найдено!")

    except Exception as e:
        error_text = f"⚠️ Произошла ошибка:\n{str(e)}\n\n" \
                    "Попробуйте отправить другое изображение или перезапустить бота"
        await update.message.reply_text(error_text)
    finally:
        # Очистка временных файлов
        for path in ["/content/temp", "/content/runs"]:
            if os.path.exists(path):
                try:
                    shutil.rmtree(path)
                except Exception as e:
                    print(f"Ошибка удаления {path}: {str(e)}")

# ==== Обработка текстовых команд ====
async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text

    if text == "📸 Отправить фото":
        await update.message.reply_text("📷 Пожалуйста, отправьте фото для анализа")
    elif text == "ℹ️ О боте":
        await about_command(update, context)
    elif text == "❓ Помощь":
        await help_command(update, context)
    elif text == "⚙️ Настройки":
        await settings_command(update, context)
    else:
        await update.message.reply_text("🤖 Я понимаю только команды из меню. Используйте кнопки ниже:")

# ==== Запуск бота ====
if __name__ == '__main__':
    # Создаем приложение
    app = ApplicationBuilder().token(BOT_TOKEN).build()

    # Добавляем обработчики
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(CommandHandler("about", about_command))
    app.add_handler(CommandHandler("settings", settings_command))

    # Обработчики текстовых команд и фото
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))
    app.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    print("🚀 Telegram-бот запущен...")
    print("Отправьте фото боту или нажмите /start")

    # Запуск бота
    app.run_polling()

🚀 Telegram-бот запущен...
Отправьте фото боту или нажмите /start


RuntimeError: Cannot close a running event loop